In [1]:
from getpass import getpass
from sqlalchemy import create_engine
import pandas as pd

from surprise import Dataset, Reader, KNNBasic, accuracy
from surprise.model_selection import KFold

In [2]:
# MySQL password
pw = getpass()

········


In [3]:
cnx = create_engine('mysql+pymysql://'+'root'+':'+pw+'@'+'localhost'+':3306/'+'yelp',echo=False)

In [4]:
restaurants = pd.read_sql('business_rest', cnx)

/Users/edgarpal/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (3719, "'utf8' is currently an alias for the character set UTF8MB3, but will be an alias for UTF8MB4 in a future release. Please consider using UTF8MB4 in order to be unambiguous.")
  result = self._query(query)


In [5]:
review = pd.read_sql('select review_id, user_id, business_id, stars from review', cnx)

In [6]:
user = pd.read_sql('select * from user', cnx)

In [7]:
reviews_rest = review.merge(restaurants, how = 'inner', on = 'business_id', suffixes = ('_rev', '_rest'))

In [8]:
reviews_rest2 = reviews_rest.merge(user, how = 'left', on = 'user_id')

In [9]:
# Percentage of reviews that are for restaurants
float(reviews_rest2.shape[0] / review.shape[0])

0.6094771448905418

In [10]:
reviews_rest2['stars_norm'] = reviews_rest2['stars_rev'] - reviews_rest2['average_stars']
reviews_rest2.head(5)

,review_id,user_id,business_id,stars_rev,address,categories,city,is_open,latitude,longitude,...,compliment_writer,cool,elite,fans,funny,name_y,review_count_y,useful,yelping_since,stars_norm
0,x7mDIiDB3jEiPGPHOmDzyw,msQe1u7Z_XuqjGoqhB0J5g,iCQpiavjjPzJ5_3gPD5Ebg,2,3708 Las Vegas Blvd S,"Pizza, Restaurants",Las Vegas,1,36.109837,-115.174212,...,0,1,None,1,1,Melissa,11,4,2011-02-24,-1.31
1,VluIpojME0yKOcRKI5L0Iw,D1_nrBr4dOrs7M82OaBRwQ,iCQpiavjjPzJ5_3gPD5Ebg,3,3708 Las Vegas Blvd S,"Pizza, Restaurants",Las Vegas,1,36.109837,-115.174212,...,0,21,None,2,31,Raed,79,104,2010-08-31,-0.99
2,nsB9JAeghk0zOaSulSm9Yw,jEjc8HxeCeI22M4di0Y1aw,iCQpiavjjPzJ5_3gPD5Ebg,5,3708 Las Vegas Blvd S,"Pizza, Restaurants",Las Vegas,1,36.109837,-115.174212,...,0,0,None,0,0,Cheryl,12,0,2016-05-22,0.83
3,pcn01EMERpCpHEcoaohdEg,RTiaaHSN80VRTLSdNK5rfQ,iCQpiavjjPzJ5_3gPD5Ebg,5,3708 Las Vegas Blvd S,"Pizza, Restaurants",Las Vegas,1,36.109837,-115.174212,...,0,1,None,0,0,Allyssa,6,3,2014-06-19,0.50
4,0mFtAPTmInbXHqDjX9eiOg,hBmJs49zYcXdV4wo8F5iiA,iCQpiavjjPzJ5_3gPD5Ebg,5,3708 Las Vegas Blvd S,"Pizza, Restaurants",Las Vegas,1,36.109837,-115.174212,...,0,0,None,0,0,Sarah,6,2,2012-12-01,1.50


In [11]:
# now limit the dataset to only frequent users
def filt_freq_users (df, low, up):
    user_review_df = df.groupby(by='user_id').count()
    frequent_posters = user_review_df[(user_review_df['business_id']>=low)&
                                      (user_review_df['business_id']<=up)
                                     ].index.values
    condensed_df = df[df['user_id'].isin(frequent_posters)]
    print('Shape of df returned: ', condensed_df.shape)
    print('Percent of rows returned: ', float(condensed_df.shape[0] / df.shape[0]))
    return condensed_df

In [12]:
# A reader is still needed but only the rating_scale param is required.
reader = Reader(rating_scale=(1, 5))
reader_norm = Reader(rating_scale=(-5, 5))

kf = KFold(n_splits=3)

In [13]:
sim_options = {'name': 'pearson', # Try cosine vs. pearson
               'user_based': False  # Try true vs. false
               }
algo = KNNBasic(sim_options=sim_options)

In [14]:
# Variations to try:
# 1-4, 5-9, 10-20, 21-30, 31-50, 51-100, 101-500, 500 - 9999
reviews_rest3 = filt_freq_users(reviews_rest2, 51, 100)

Shape of df returned:  (314062, 37)
Percent of rows returned:  0.08592592349396667


In [15]:
# The columns must correspond to user id, item id and ratings (in that order).
# data = Dataset.load_from_df(reviews_rest3[['user_id', 'business_id', 'stars_rev']], reader)

In [ ]:
# Normalized version (subtracting average stars from user)
data_norm = Dataset.load_from_df(reviews_rest3[['user_id', 'business_id', 'stars_norm']], reader_norm)

In [ ]:
for trainset, testset in kf.split(data_norm):

    # train and test algorithm.
    algo.fit(trainset)
    predictions = algo.test(testset)

    # Compute and print Root Mean Squared Error
    accuracy.rmse(predictions, verbose=True)

Computing the pearson similarity matrix...


/Users/edgarpal/anaconda3/lib/python3.6/site-packages/surprise/prediction_algorithms/algo_base.py:293: RuntimeWarning: invalid value encountered in sqrt
  sim = construction_func[name](*args)


Done computing similarity matrix.
RMSE: 1.1747
